In [1]:
import cv2
import numpy as np
import os

# Constants
NUM_FRAMES = 16  # Fixed number of frames per segment

def extract_fixed_frame_segments(video_path, annotations, img_height, img_width):
    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    video_name = os.path.basename(video_path)
    
    labeled_segments = []  # To store segments as (frames, label)

    # Process each annotated segment
    for segment in annotations.get(video_name, []):
        start_frame = int(segment['start'] * fps)
        end_frame = int(segment['end'] * fps)
        num_segment_frames = end_frame - start_frame
        
        # Initialize frame list for this segment
        frames = []
        cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)
        
        # Extract frames from the segment
        for _ in range(num_segment_frames):
            ret, frame = cap.read()
            if not ret:
                break

            # Resize and normalize frame
            frame_resized = cv2.resize(frame, (img_width, img_height))
            frame_resized = frame_resized / 255.0  # Normalize to [0, 1]
            frames.append(frame_resized)

        # Handle cases where the number of frames < NUM_FRAMES
        if len(frames) > NUM_FRAMES:
            # Uniformly sample 16 frames if more than 16 frames
            step = len(frames) // NUM_FRAMES
            frames = frames[::step][:NUM_FRAMES]
        elif len(frames) < NUM_FRAMES:
            # Pad with last frame if fewer than 16 frames
            frames.extend([frames[-1]] * (NUM_FRAMES - len(frames)))
        
        # Store the labeled segment
        label = 1 if segment else 0  # 1 for anomalous, 0 for normal
        labeled_segments.append((np.array(frames), label))

    cap.release()
    return labeled_segments
